In [1]:
%%time
# Import the relevant modules to be used later
from __future__ import print_function
import gzip
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import struct
import sys

try: 
    from urllib.request import urlretrieve 
except ImportError: 
    from urllib import urlretrieve

# Config matplotlib for inline plotting
%matplotlib inline

**Data Prep**
- Load labels and read 64x64 image files into pixel data

In [64]:
from PIL import Image
def loadImage(name):
    filename = "data/train/" + name + ".png"
    img = Image.open(filename)
    pixels = img.load()
    img_data = ''
    for i in range(img.size[0]):    # for every col:
        for j in range(img.size[1]):    # For every row
            img_data = img_data + ' ' + str(pixels[i,j])
    return img_data


In [65]:
%%time
#if os.path.isfile("data/train_labels.csv")
import csv
count = 0
larray = ['1 0 0 0', '0 1 0 0', '0 0 1 0', '0 0 0 1']
label = ''
feature = ''
output = []
with open('data/train_labels.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if count > 0:
            #output = '({0}) file: {1} label: {2}'.format(count, row[0], row[1]) + ' onehot: ' + larray[int(row[1])-1] + '\n'
            label = larray[int(row[1])-1]
            feature = loadImage(row[0])
            output.append('|labels {} |features {}\n'.format(label, feature))
        count = count + 1

import time
import datetime
filename = 'data/Train-64x64x20000_' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S') + '.txt'
if not os.path.isfile(filename):
    print("Saving", filename )
    with open(filename, 'w') as f:
        for row in output:
            f.write(row)
    print("File save complete.")
else:
    print("File already exists", filename)

Saving data/Train-64x64x20000_20180717_230207.txt
File save complete.
Wall time: 1min 50s


In [62]:
import time
import datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
print(st)

20180717_224608
